In [ ]:
from taxbrain.taxbrain import TaxBrain
from taxbrain import report
import pandas as pd

None
/opt/homebrew/anaconda3/envs/taxcalc-dev/lib/python312.zip
/opt/homebrew/anaconda3/envs/taxcalc-dev/lib/python3.12
/opt/homebrew/anaconda3/envs/taxcalc-dev/lib/python3.12/lib-dynload

/opt/homebrew/anaconda3/envs/taxcalc-dev/lib/python3.12/site-packages
__editable__.taxcalc-4.6.0.finder.__path_hook__


In [ ]:
tcja_ext = "https://raw.githubusercontent.com/PSLmodels/Tax-Calculator/master/taxcalc/reforms/ext.json"

single_amount = 7250
sd_senior_reform = {
    "STD_Aged": {"2026":  [single_amount, single_amount*2, single_amount, single_amount, single_amount*2]}
}

In [3]:
dynamic = TaxBrain(2026, 2026, microdata="TMD", base_policy=tcja_ext, reform=sd_senior_reform, behavior={"sub": 0.25, "cg": -3.45})
dynamic.run()

ValueError: growfactors file taxcalc/tmd_growfactors.csv does not exist

In [ ]:
df_base = dynamic.base_calc[['s006', 'MARS', 'age_head', 'age_spouse', 'mtr_combined']]
df_reform = dynamic.reform_calc[['s006', 'MARS', 'age_head', 'age_spouse', 'mtr_combined']]

In [ ]:
def my_agg(x):
    if x['MARS'].iloc[0]==2:
        y = x[(x['age_head'] >= 65) | (x['age_spouse'] >= 65)]
    else:
        y = x[(x['age_head'] >= 65)]
    y['mtr_total'] = y['s006']*y['mtr_combined']
    mtr_avg = sum(y['mtr_total'])/sum(y['s006'])
    names = {
        'mtr_avg': f"{mtr_avg:.1f}"
    }
    return pd.Series(names)

In [ ]:
df_base_out = df_base.groupby(['MARS'], as_index=True).apply(my_agg).reset_index()
df_reform_out = df_reform.groupby(['MARS'], as_index=True).apply(my_agg).reset_index()

/var/folders/8d/mdshyj_x3x5bn5cmqhrhn3yh0000gn/T/ipykernel_57654/3903910536.py:1: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  df_base_out = df_base.groupby(['MARS'], as_index=True).apply(my_agg).reset_index()
/var/folders/8d/mdshyj_x3x5bn5cmqhrhn3yh0000gn/T/ipykernel_57654/3903910536.py:2: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  df_reform_out = df_reform.groupby(['MARS'], as_index=True).apply(my_agg

In [ ]:
df_combined = pd.concat([df_base_out, df_reform_out['mtr_avg']], axis=1)
mapping = {1: "single", 2: "joint", 3: "separate", 4: "household-head", 5: "widow(er)"}
df_combined['MARS'] = df_combined['MARS'].map(mapping)
df_combined.columns = ['Filing Status', 'Average Marginal Tax Rate (%, Baseline)', 'Average Marginal Tax Rate (%, Reform)']

In [ ]:
df_combined.to_excel('SD_Increase_for_Seniors.xlsx', index=False)
df_combined.to_csv('SD_Increase_for_Seniors.csv', index=False)